 # 1. Introduction and Defining Business Problem:

Berlin, It is a place where I am staying right now. In terms of population and area, Berlin is the largest city and capital for Germany. Berlin is the second famous startup hub in Europe.

## Problem Statement: Prospects for opening a food Restaurant or a new office in Berlin.

During the daytime, especially during lunch hours, office areas provide huge opportunities for restaurants.  An average meal priced (one lunch meal 5€). The shops are usually always full during lunch hours (11 am — 2-30 pm. Given this scenario, I will be showing the benefits and pitfalls of opening a restaurant in highly dense office places including office delivery. However, I am unaware of the profit, I do believe there will be huge benefits by opening a restaurant in the dense area of companies. I will be covering the top 10 places in Berlin.

Additionally, by understanding the type of companies that are located in each area will result in valuable information for opening a new office. Such as finding an area relatively less cost for opening an office and public transportation analysis to reach this company.

## Description of the data and solving the problems

To solve the above problems, a location-based dataset is important. However, neighther it is not available directly in internet nor from the Foursquare website. Hence, I decided to scrape the required data. 

There are 2 datasets:
    
    Company-related data with following columns:
        1. Company name: In Germany, a company is characterized as mbH, GmbH, AG, AG &Co.
        2. Address: It is composed of a street name, GPS coordinates, zip code, neighborhood
        3. Category: It is a type of company i.e software company, construction company e.t.c

    Berlin geographical dataset is composed with the following columns:
        1. Zip code
        2. Neighborhood
        3. District



### To solve above problems the following questions needed to answer.


    1. How many companies are located in each area? (i.e per zip code)
    2. Which type of shops are located near to each company? (i.e with the help GPS coordinates of a company). This particular question will be answered through Foursquare data.
    3. How similar each company in terms of its services?
    4. Possible public transportation services to reach the company?
    5. How far the residential area is located? 
    6. What are the average rental prices for a given area? By analyzing clusters of companys that are located for an area. (i.e This question will be answered if the data relevant data is available else will be skipped.)  


As a part of the report, I will walk you through each step of this project and address them separately. These answeres will justify a better place to open a restaurant or to open an office for my stakeholders. 

Let me, show you an outline of the initial data preparation. 



### Target Audience

#### What type of clients or a group of people would be interested in this project?

    1.For data scientists, who wnats to do exploratory data analysis techniques to obtain necessary data, analyze it, and, finally be able to tell a story out of it.
    2.Business personnel who wants to invest or open a restaurant. This analysis will be a comprehensive guide to start or expand restaurants targeting the large pool of office places during lunch hours.
    3. Furthermore, the analysis of company locations in Berlin will hugly benifit the business personnel for opening their new office.

# 2. Preprocessing the dataset  having companies list, postal codes and neighborhoods in Berlin

In [1]:
import wikipedia
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import unicodedata
import requests
import json

### Extract Berlin neighborhoods and boroughs from wiki using BeautifulSoup

#### If like to scrape the data, run the following block of code.

```python
wiki_page = "https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins"

html_ = requests.get(wiki_page)
soup = BeautifulSoup(html_.content, "html.parser")
table = soup.find_all("table")
data_table = table[2]

data_nei_ = []
columns = ["Nr","Ortteil","Bezirk","Flaeche","Einwohner_30_07_2019","Einwohner_km2"] 

for tr in data_table.find_all("tr")[1:]:
    data = []
    for td in tr.find_all("td"):
        data.append(td.get_text().strip())
    data_nei_.append(data)
data_nei_
df_nei = pd.DataFrame(data_nei_, columns=columns)
df_nei
```

### Step 2.1: Load dataset having district and its neighborhoods

In [2]:
berlin_neighborhoods = pd.read_csv("data/berlin_places.csv")
berlin_neighborhoods = berlin_neighborhoods[["Ortsteil","Bezirk"]]
berlin_neighborhoods["Ortsteil"] = berlin_neighborhoods["Ortsteil"].str.strip()
berlin_neighborhoods["Bezirk"] = berlin_neighborhoods["Bezirk"].str.strip()

### Step 2.2: Getting Coordinates of Major Districts(i.e Bezirk) : Geopy Client

##### if you like to get berlin district codes run the following code

```python
from geopy.geocoders import Nominatim
import json 
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim(user_agent="a_explorer")

def explore(x):
    ll = geolocator.geocode(x)
    return (ll.latitude, ll.longitude)

def do_geocode(x):
    try:
        return explore(x)
    except GeocoderTimedOut:
        return explore(x)


unique_bezirk = berlin_neighborhoods.Bezirk.unique()
# unique_ortteil = berlin_neighborhoods.Ortsteil.unique()
# unique_ortteil = [i.strip() for i in unique_ortteil]

ll = {}
for item in unique_bezirk:
    x,y = do_geocode(item)
    ll[item] = (x,y)
ll
```

In [3]:
with open("data/bezirkgeo.json", "r") as f:
    geo_district = json.load(f)

berlin_neighborhoods["Bezirkgeo"] = berlin_neighborhoods.Bezirk.apply(lambda x: geo_district[x])
berlin_neighborhoods.head()

,Ortsteil,Bezirk,Bezirkgeo
0,Mitte,Mitte,"[52.5176896, 13.4023757]"
1,Moabit,Mitte,"[52.5176896, 13.4023757]"
2,Hansaviertel,Mitte,"[52.5176896, 13.4023757]"
3,Tiergarten,Mitte,"[52.5176896, 13.4023757]"
4,Wedding,Mitte,"[52.5176896, 13.4023757]"


### Step 2.3: Load the daset having neighborhood and its postal codes

In [4]:
berlin_postalcodes = pd.read_excel("data/Bundesland Berlin.xlsx")
berlin_postalcodes[berlin_postalcodes.Ortsteil=="Wedding"]

,PLZ,Ortsteil
241,13347,Wedding
243,13349,Wedding
244,13351,Wedding
246,13353,Wedding
249,13357,Wedding
251,13359,Wedding
256,13405,Wedding
259,13407,Wedding
262,13409,Wedding


### Step 2.4: Load the dataset having company and its address, category of comapny, gps coordinates. 

###### Scraping of such data is still in progress and will be updated soon...........

In [5]:
profiles = pd.read_csv("data/company_profile.csv")
profiles = profiles[["url","location","info"]]
profiles['location'] = profiles['location'].apply(lambda x: "{:.3f}".format(x) if not pd.isnull(x) else x)

In [6]:
profiles['location'] = profiles['location'].astype(str)
profiles['info'] = profiles['info'].astype(str)
cleaned_profiles = profiles.groupby(["url"])["info"].agg([('info', ','.join)])
cleaned_location = profiles.groupby(["url"])["location"].agg([('location', ','.join)])
cleaned_profiles["profile"] = cleaned_profiles["info"]+ "," +cleaned_location["location"]

In [7]:
def correct(x):
    cleaned_str = []
    for i in x.split(","):
        if i != 'nan':
            cleaned_str.append(i)
    return cleaned_str

cleaned_profiles["profile"] = cleaned_profiles.profile.apply(lambda x: correct(x))
cleaned_profiles.head()

,info,profile
url,,
(KA) Kraft Automobile GmbH,"nan,nan,(KA) Kraft Automobile GmbH,Wexstrasse ...","[(KA) Kraft Automobile GmbH, Wexstrasse 15, 10..."
(know:bodies) gesellschaft für integrierte kommunikation und bildungsberatung mbh,"nan,nan,(know:bodies) gesellschaft für integri...",[(know:bodies) gesellschaft für integrierte ko...
07schanksysteme gmbh,"nan,nan,07schanksysteme gmbh,Sprengelstrasse 1...","[07schanksysteme gmbh, Sprengelstrasse 15, 133..."
0815-Industries KG,"nan,nan,0815-Industries KG,Feldtmannstrasse 15...","[0815-Industries KG, Feldtmannstrasse 152, 130..."
1 Berlin x Hausverwaltung GmbH + Co. KG,"1 Berlin x Hausverwaltung GmbH + Co. KG,Königi...","[1 Berlin x Hausverwaltung GmbH + Co. KG, Köni..."


In [8]:
profiles_list = cleaned_profiles["profile"].values.tolist()
company_data = pd.DataFrame(profiles_list)[[0,1,2,3,4,5]]
company_data.columns= ["Name", "Street","Zipcode","City","Lat","Lan"]
print("Shape of data: ",company_data.shape)
company_data[company_data.isnull().any(axis=1)].head()

Shape of data:  (6119, 6)


,Name,Street,Zipcode,City,Lat,Lan
4,1 Berlin x Hausverwaltung GmbH + Co. KG,Königin-Luise-Strasse 43,14195,Berlin,None,None
5,1 Berlin x Recon GmbH + Co. KG,Königin-Luise-Strasse 43,14195,Berlin,None,None
6,1 Berlin x Recon Holding GmbH,Königin-Luise-Strasse 43,14195,Berlin,None,None
20,2 + 1 Global Projects GmbH,Chausseetrasse 14,10115,Berlin,None,None
22,2019 Holding GmbH,Waghäuseler Strasse 10 e,10715,Berlin,None,None


In [9]:
# droping data which does't have Latitude an longitude
company_data = company_data.dropna()
company_data = company_data.reset_index(drop=True)
company_data.head()

,Name,Street,Zipcode,City,Lat,Lan
0,(KA) Kraft Automobile GmbH,Wexstrasse 15,10715,Berlin,52.479,13.336
1,(know:bodies) gesellschaft für integrierte kom...,Sophie-Charlotten-Strasse 103,14059,Berlin,52.518,13.287
2,07schanksysteme gmbh,Sprengelstrasse 15,13353,Berlin,52.542,13.355
3,0815-Industries KG,Feldtmannstrasse 152,13088,Berlin,52.564,13.474
4,1-2-3 Beschläge GmbH,Colditzstrasse 33,12099,Berlin,52.455,13.396


In [10]:
unique_ortteil = berlin_neighborhoods.Ortsteil.unique()
unique_ortteil = [i.strip() for i in unique_ortteil]

### Step 2.5: Load the dataset having avarage rental price per Ortsteil

In [11]:
avgprice = pd.read_csv("data/prices.csv") 
# preprocess data 
avg1 = avgprice["text"][0]
avg1 = list(filter(None, [item.strip() for item in avg1.split("\n")]))

avg2 = avgprice["text"][1]
avg2 = list(filter(None, [item.strip() for item in avg2.split("\n")]))

In [12]:
def dataframe(x):
    y = np.array(x)
    y = y.reshape(-1,2)
    df = pd.DataFrame(y[1:], columns=y[0])
    df["Ortsteil"] = df["STADTTEIL"].apply(lambda x : x.split(" (")[0])
    return df

In [13]:
df_avg1 = dataframe(avg1)
df_avg1.head()

,STADTTEIL,€/m²,Ortsteil
0,Lichterfelde (Steglitz),"11,20 €",Lichterfelde
1,Mahlsdorf (Hellersdorf),"10,36 €",Mahlsdorf
2,Mariendorf (Tempelhof),"10,90 €",Mariendorf
3,Marienfelde (Tempelhof),"19,82 €",Marienfelde
4,Märkisches Viertel,"8,22 €",Märkisches Viertel


In [14]:
df_avg2 = dataframe(avg2)
df_avg2.head()

,STADTTEIL,€/m²,Ortsteil
0,Adlershof (Treptow),"12,51 €",Adlershof
1,Alt-Hohenschönhausen (Hohenschönhausen),"10,76 €",Alt-Hohenschönhausen
2,Alt-Treptow,"12,90 €",Alt-Treptow
3,Altglienicke (Treptow),"9,95 €",Altglienicke
4,Baumschulenweg (Treptow),"10,82 €",Baumschulenweg


In [15]:
Ortsteil_price = pd.concat([df_avg1, df_avg2]).reset_index(drop=True)
Ortsteil_price

,STADTTEIL,€/m²,Ortsteil
0,Lichterfelde (Steglitz),"11,20 €",Lichterfelde
1,Mahlsdorf (Hellersdorf),"10,36 €",Mahlsdorf
2,Mariendorf (Tempelhof),"10,90 €",Mariendorf
3,Marienfelde (Tempelhof),"19,82 €",Marienfelde
4,Märkisches Viertel,"8,22 €",Märkisches Viertel
...,...,...,...
81,Köpenick (Köpenick),"11,62 €",Köpenick
82,Kreuzberg (Kreuzberg),"16,54 €",Kreuzberg
83,Lankwitz (Steglitz),"10,07 €",Lankwitz
84,Lichtenberg (Lichtenberg),"11,60 €",Lichtenberg


### Step 2.6: Using Foursquare location data for each company location 

Preparing data for each company is a big deal. However, I woul like to skip this process just for time beeing. I will come back again to this step after having a good understanding of company data.

### Step 2.7: Combining all datasets except company dataset

In [16]:
Ortsteil_prices = Ortsteil_price[["Ortsteil","€/m²"]].set_index("Ortsteil")
dict_ortprice = Ortsteil_prices.to_dict()["€/m²"]

In [17]:
def insert_price(x):
    x = x.strip()
    if x in dict_ortprice.keys():
        x = dict_ortprice[x]
        x = x.split()[0]
        x = x.replace(",",".")
    else:
        x = 0
    return x
berlin_neighborhoods["€/m²"] = berlin_neighborhoods["Ortsteil"].apply(lambda x : insert_price(x))
berlin_neighborhoods["€/m²"] = berlin_neighborhoods["€/m²"].astype(float)
berlin_neighborhoods.head()

,Ortsteil,Bezirk,Bezirkgeo,€/m²
0,Mitte,Mitte,"[52.5176896, 13.4023757]",17.27
1,Moabit,Mitte,"[52.5176896, 13.4023757]",17.59
2,Hansaviertel,Mitte,"[52.5176896, 13.4023757]",16.55
3,Tiergarten,Mitte,"[52.5176896, 13.4023757]",16.04
4,Wedding,Mitte,"[52.5176896, 13.4023757]",11.33


In [18]:
berlin_postalcodes = berlin_postalcodes.set_index("Ortsteil")
berlin_neighborhoods = berlin_neighborhoods.set_index("Ortsteil")

In [19]:
merged = berlin_postalcodes.merge(berlin_neighborhoods, left_index=True, right_index=True)

In [20]:
merged

,PLZ,Bezirk,Bezirkgeo,€/m²
Ortsteil,,,,
Adlershof,12487,Treptow-Köpenick,"[52.417893, 13.6001848]",12.51
Adlershof,12489,Treptow-Köpenick,"[52.417893, 13.6001848]",12.51
Alt-Hohenschönhausen,13051,Lichtenberg,"[52.534306, 13.502326]",10.76
Alt-Hohenschönhausen,13053,Lichtenberg,"[52.534306, 13.502326]",10.76
Alt-Hohenschönhausen,13055,Lichtenberg,"[52.534306, 13.502326]",10.76
...,...,...,...,...
Zehlendorf,14129,Steglitz-Zehlendorf,"[52.4292052, 13.2299741]",11.89
Zehlendorf,14163,Steglitz-Zehlendorf,"[52.4292052, 13.2299741]",11.89
Zehlendorf,14165,Steglitz-Zehlendorf,"[52.4292052, 13.2299741]",11.89


### 2.7.1 replacing zero values of rent with the mean values of corresponding district. 

In [21]:
print("Districts having no avarage rent values: ")
merged[merged["€/m²"]==0].Bezirk.unique().tolist()

Districts having no avarage rent values: 


['Pankow',
 'Charlottenburg-Wilmersdorf',
 'Lichtenberg',
 'Spandau',
 'Reinickendorf']

In [22]:
# evaluating mean values for zero rent values of district.
mean_lichtenberg = merged[(merged["Bezirk"]=='Lichtenberg') & (merged["€/m²"] !=0)]["€/m²"].mean()
mean_pankow = merged[(merged["Bezirk"]=='Pankow') & (merged["€/m²"] !=0)]["€/m²"].mean()
mean_charwil = merged[(merged["Bezirk"]=='Charlottenburg-Wilmersdorf') & (merged["€/m²"] !=0)]["€/m²"].mean()
mean_spandau = merged[(merged["Bezirk"]=='Spandau') & (merged["€/m²"] !=0)]["€/m²"].mean()
mean_reini = merged[(merged["Bezirk"]=='Reinickendorf') & (merged["€/m²"] !=0)]["€/m²"].mean()

In [23]:
merged[merged["Bezirk"]=='Spandau']

,PLZ,Bezirk,Bezirkgeo,€/m²
Ortsteil,,,,
Falkenhagener Feld,13583,Spandau,"[52.535788, 13.1977924]",0.00
Falkenhagener Feld,13585,Spandau,"[52.535788, 13.1977924]",0.00
Falkenhagener Feld,13589,Spandau,"[52.535788, 13.1977924]",0.00
Falkenhagener Feld,13591,Spandau,"[52.535788, 13.1977924]",0.00
Gatow,14089,Spandau,"[52.535788, 13.1977924]",0.00
Hakenfelde,13585,Spandau,"[52.535788, 13.1977924]",12.37
Hakenfelde,13587,Spandau,"[52.535788, 13.1977924]",12.37
Hakenfelde,13589,Spandau,"[52.535788, 13.1977924]",12.37
Haselhorst,13599,Spandau,"[52.535788, 13.1977924]",8.07


In [24]:
# replacing zero values with the mean values.
merged.loc[(merged["Bezirk"]=='Pankow')&(merged["€/m²"]==0),"€/m²"]= mean_pankow
merged.loc[(merged["Bezirk"]=='Spandau')&(merged["€/m²"]==0),"€/m²"]= mean_spandau
merged[merged["Bezirk"]=='Spandau']

,PLZ,Bezirk,Bezirkgeo,€/m²
Ortsteil,,,,
Falkenhagener Feld,13583,Spandau,"[52.535788, 13.1977924]",10.070588
Falkenhagener Feld,13585,Spandau,"[52.535788, 13.1977924]",10.070588
Falkenhagener Feld,13589,Spandau,"[52.535788, 13.1977924]",10.070588
Falkenhagener Feld,13591,Spandau,"[52.535788, 13.1977924]",10.070588
Gatow,14089,Spandau,"[52.535788, 13.1977924]",10.070588
Hakenfelde,13585,Spandau,"[52.535788, 13.1977924]",12.370000
Hakenfelde,13587,Spandau,"[52.535788, 13.1977924]",12.370000
Hakenfelde,13589,Spandau,"[52.535788, 13.1977924]",12.370000
Haselhorst,13599,Spandau,"[52.535788, 13.1977924]",8.070000


In [25]:
merged.loc[(merged["Bezirk"]=='Lichtenberg')&(merged["€/m²"]==0),"€/m²"]= mean_lichtenberg
merged.loc[(merged["Bezirk"]=='Charlottenburg-Wilmersdorf')&(merged["€/m²"]==0),"€/m²"]= mean_charwil
merged.loc[(merged["Bezirk"]=='Reinickendorf')&(merged["€/m²"]==0),"€/m²"]= mean_reini
merged.head()

,PLZ,Bezirk,Bezirkgeo,€/m²
Ortsteil,,,,
Adlershof,12487,Treptow-Köpenick,"[52.417893, 13.6001848]",12.51
Adlershof,12489,Treptow-Köpenick,"[52.417893, 13.6001848]",12.51
Alt-Hohenschönhausen,13051,Lichtenberg,"[52.534306, 13.502326]",10.76
Alt-Hohenschönhausen,13053,Lichtenberg,"[52.534306, 13.502326]",10.76
Alt-Hohenschönhausen,13055,Lichtenberg,"[52.534306, 13.502326]",10.76


In [26]:
merged.loc["Wedding"]

,PLZ,Bezirk,Bezirkgeo,€/m²
Ortsteil,,,,
Wedding,13347,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13349,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13351,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13353,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13357,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13359,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13405,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13407,Mitte,"[52.5176896, 13.4023757]",11.33
Wedding,13409,Mitte,"[52.5176896, 13.4023757]",11.33
